In [1]:
import pandas as pd
import keras
import keras_hub
import tensorflow as tf
import os

In [2]:
df_ft = pd.read_excel(
    "/content/drive/Shareddrives/Capstone1/Capstone II/Data for FT/final_ft_eval.xlsx",
    sheet_name='fine_tuning_set' # Ensure this is the sheet with your fine-tuning data
)

In [3]:
df_ft.head()

,id,question,answer,source,mode of generation
0,1,"In Texas marriage law, what is the default pol...",The policy is to preserve and uphold each marr...,fa.1.pdf,manual
1,2,"In Texas marriage law, what is the default pol...",Marriages are presumed valid; only strong stat...,fa.1.pdf,manual
2,3,"In Texas marriage law, what is the default pol...","By default, Texas treats marriages as valid un...",fa.1.pdf,manual
3,4,"In Texas marriage law, what is the default pol...",Validity is presumed; only compelling statutor...,fa.1.pdf,manual
4,5,If a person has multiple alleged marriages to ...,The most recent marriage is presumed valid unl...,fa.1.pdf,manual


In [4]:
# Assuming you have df_ft loaded from your Excel file...

# 1. Define the persona instruction separately
persona_instruction = (
    "You are a highly experienced and cautious **Texas Family Law Expert**."
    "Your primary goal is to provide a brief but legally rigorous, and factually accurate "
    "responses to the legal questions. The answer should be complete and cater to the main question asked."
)

# 2. Extract the prompts and responses into two separate lists

# The 'prompt' is the combined persona + instruction, ending right before the model's response starts.
prompts = []
# The 'response' is just the ground truth answer (the label the model should generate).
responses = []

# for index, row in df_ft.iterrows():
#     # Construct the instruction template part (the prompt)
#     prompt_text = (
#         f"{persona_instruction}\n\n"
#         f"Instruction:\n{row['question']}\n\n"
#         f"Response:\n"  # Note the empty response tag to tell the model where to start
#     )

#     prompts.append(prompt_text)
#     responses.append(row['answer'])

# The actual structure the model was trained on:
START_TURN_USER = "<start_of_turn>user\n"
END_TURN_USER = "<end_of_turn>"
START_TURN_MODEL = "<start_of_turn>model\n"
END_TURN_MODEL = "<end_of_turn>"

for index, row in df_ft.iterrows():
    # Combine the persona and question into the 'user' turn
    full_prompt = (
        f"{persona_instruction}\n\n"
        f"Instruction:\n{row['question']}"
    )

    # The full sequence for training is: [USER TURN] + [MODEL TURN]
    prompt_text = (
        f"{START_TURN_USER}{full_prompt}{END_TURN_USER}"
        f"{START_TURN_MODEL}" # This tells the model where to start generating
    )

    # The full label includes the answer and the turn end tag
    response_text = f"{row['answer']}{END_TURN_MODEL}"

    prompts.append(prompt_text)
    responses.append(response_text)


# 3. Create the training dictionary structure
ft_dict = {
    "prompts": prompts,
    "responses": responses
}

# 4. Convert the dictionary to a TensorFlow Dataset
# This creates a dataset where each element is a dictionary with keys "prompts" and "responses"
batch_size = 1
train_ds = tf.data.Dataset.from_tensor_slices(ft_dict).batch(batch_size)

In [5]:
os.environ["KERAS_BACKEND"] = "jax" # Or "tensorflow" or "torch"

# 2. Configure mixed precision (recommended for speed and memory efficiency)
keras.config.set_floatx("bfloat16")

# 3. Load the model and its preprocessor
# 1. Load the model with bfloat16
gemma_lm = keras_hub.models.Gemma3CausalLM.from_preset("gemma3_instruct_4b", dtype="bfloat16")
gemma_lm.preprocessor.sequence_length = 1024

# 2. 🚨 ADD THIS LINE BEFORE ENABLING LoRA 🚨
# gemma_lm.quantize("int8")

# 4. Enable LoRA on the backbone
# Rank (r) controls the number of new trainable parameters. 4, 8, or 16 are typical starting points.
# We'll use 4 for minimal resource usage.
gemma_lm.backbone.enable_lora(rank=32)

# Print summary to see the massive reduction in trainable parameters
print(gemma_lm.summary())

100%|██████████| 1.84k/1.84k [00:00<00:00, 4.04MB/s]


100%|██████████| 5.55k/5.55k [00:00<00:00, 9.90MB/s]


100%|██████████| 4.47M/4.47M [00:01<00:00, 2.54MB/s]


100%|██████████| 8.79G/8.79G [09:38<00:00, 16.3MB/s]


Preprocessor: "gemma3_causal_lm_preprocessor"

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ images (InputLayer)           │ (None, None, 896, 896, 3) │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ vision_indices (InputLayer)   │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ vision_mask (InputLayer)      │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma3_backbone               │ (None, 256, 2560)         │   4,337,877,360 │ images[0][0],              │
│ (Gemma3Backbone)              │                           │                 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0],           │
│                               │                           │                 │ vision_indices[0][0],      │
│                               │                           │                 │ vision_mask[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, 256, 262144)       │     671,088,640 │ gemma3_backbone[0][0]      │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 4,337,877,360 (8.87 GB)

 Trainable params: 38,023,936 (80.24 MB)

 Non-trainable params: 4,299,853,424 (8.79 GB)

None


In [6]:
import math

# 1. Calculate the total number of steps using the length of the 'prompts' list

# --- 2. Calculate the correct steps_per_epoch ---
total_samples = len(prompts)
total_steps = math.ceil(total_samples / batch_size)

# Use a low learning rate specific for fine-tuning LLMs
optimizer = keras.optimizers.AdamW(
    learning_rate=1e-4,
    gradient_accumulation_steps=8
)

# Compile the model
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

print("Starting Fine-Tuning...")

# Run the training process
gemma_lm.fit(
    train_ds,
    epochs=1,
    # 'train_ds' is already batched, so no 'batch_size' needed
    steps_per_epoch=total_steps # This now uses the correctly defined variable
)

print("Fine-Tuning Complete!")

Starting Fine-Tuning...
4154/4154 ━━━━━━━━━━━━━━━━━━━━ 738s 133ms/step - loss: 3.2360 - sparse_categorical_accuracy: 0.4228
Fine-Tuning Complete!


In [10]:
# Define a descriptive file name for your LoRA weights
save_path = "/content/drive/MyDrive/Colab Notebooks/Capstone/gemma_4b_lora_weights_v6.weights.h5"

# Save ONLY the trainable (LoRA) weights
gemma_lm.save_weights(save_path)

print(f"✅ Fine-tuned LoRA weights successfully saved to: {save_path}")

✅ Fine-tuned LoRA weights successfully saved to: /content/drive/MyDrive/Colab Notebooks/Capstone/gemma_4b_lora_weights_v6.weights.h5
